# Pre-requisites

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

In [44]:
import os
os.environ['AWS_PROFILE'] = 'hcls-agents'

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

In [46]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [48]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "VariantInterpreter"
agent_name = f"{stack_name}-Agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBRkExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for interpreting annotated variant files and answering user Questions"
agent_instruction = "You are an expert at analyzing genomics variants with VEP. You have access to a set of tools to help analyze genetic variants for patient including total variants, variants per chromosome, pathogenic variants and the VEP report file.  You have access to two action group functions: retrieve_existing_vep_report and vep_feature_extraction. Always use vep_feature_extraction to provide variants feature summary or gene level summary. Prioritize usin using featue extraction function. Please analyze this data and prepare to answer questions: 1. what is the total variant count in the patient id: “3186764”?; 2. howmany variants are present in chromosome 1?; 3. which chromosome has stronger presence of high-impact variants?; 4. what are those top 5 genes have highest number of high-impact variants?; 5. what is the impact distribution of variants in chromosome 5?; 6. howmany splice region variants are present in STAT3 gene?; 7. which gene has highest stop gained or lost variants?; 8. which gene has highest Non-coding RNA variants?; 9. which are those top 3 genes I should focus for this patient based on variant affecting proteins functional loss?; 10. Are there any SIFT scores available for these top 3 genes?"
agent_action_group_name = "variant-inter-actionGroup"
agent_action_group_description = "Actions for Retrieving information from variant interpretations"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{lambda_function_name}"
#lambda_function_arn = lambda_client.get_function(FunctionName=lambda_function_name)['Configuration']['FunctionArn']

# Create Lambda Function 

In [ ]:
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
print(lambda_function_name)

# attach the s3 full access policy to the Lambda function role
s3_full_access_policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
bedrock_policy_arn = "arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
AWSLambdaBasicExecutionRole_arn = "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"

# create role for the Lambda function
try:
    iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }),
        Description="Role for Lambda function to access S3 and other resources"
    )

    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=s3_full_access_policy_arn
        
    )

except Exception as e:
    print("Error creating role: ", e)
    print("Role already exists. Continuing...")
    

In [49]:
Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn']
print(lambda_function_role)

VariantInterpreter-LambdaExecutionRole


In [51]:
iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=AWSLambdaBasicExecutionRole_arn
    )
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=bedrock_policy_arn
)

{'ResponseMetadata': {'RequestId': '574bd221-1b82-4c55-85b8-0424f8da97b2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 05:10:30 GMT',
   'x-amzn-requestid': '574bd221-1b82-4c55-85b8-0424f8da97b2',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [ ]:
# Package up the lambda function code and deploy to Lambda function
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("LambdaAgent/lambda_function.py", arcname="lambda_function.py")
z.close()
zip_content = s.getvalue()

role_response = iam_client.get_role(RoleName=lambda_function_role)
lambda_function_role = role_response['Role']['Arn']

response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',  # updated to Python 3.12
    Role=lambda_function_role,
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': zip_content
    },
    Timeout=600,  # 10 minutes = 600 seconds
    MemorySize=400  # Memory in MB
)

In [ ]:
#response = lambda_client.update_function_configuration(
#    FunctionName=lambda_function_name,
#    Handler='lambda_function.lambda_handler'
#)

#response = lambda_client.update_function_configuration(
#     FunctionName=lambda_function_name,
#     Timeout=600,  # 10 minutes = 600 seconds
#     MemorySize=400  # Memory in MB
#)

# Create Agent

In [52]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except Exception:
    print(f"Policy {agent_bedrock_allow_policy_name} already exists")

In [53]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )


    # Pause to make sure role is created
    time.sleep(10)
        
    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
except Exception as e:
    print(f"Error creating role: {e}. Make sure it doesnt exist already")

In [54]:
try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print(response)

except:
    print("Agent already exists, skipping creation")
    agent_info = [agent for agent in bedrock_agent_client.list_agents()['agentSummaries'] if agent['agentName']==agent_name][0]
    print(agent_info)
    agent_id = agent_info['agentId']
    agent_version = bedrock_agent_client.list_agent_versions(agentId=agent_id)['agentVersionSummaries'][0]['agentVersion']
    response = bedrock_agent_client.list_agent_action_groups(
        agentId=agent_id,
        agentVersion=agent_version
    )
    print(response['actionGroupSummaries'][0])
    action_group_id = response['actionGroupSummaries'][0]['actionGroupId']
    print(f"Using agent_id {agent_id} and action_group_id {action_group_id}")

{'ResponseMetadata': {'RequestId': '9c3fd060-9577-42a4-8b2a-3b8200a5cf03', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 05:19:02 GMT', 'content-type': 'application/json', 'content-length': '1851', 'connection': 'keep-alive', 'x-amzn-requestid': '9c3fd060-9577-42a4-8b2a-3b8200a5cf03', 'x-amz-apigw-id': 'JnXvGGM8IAMES4w=', 'x-amzn-trace-id': 'Root=1-680c6cc6-2e44f98059c394852afe8dbc'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:735766051544:agent/BLUBEL66XB', 'agentCollaboration': 'DISABLED', 'agentId': 'BLUBEL66XB', 'agentName': 'VariantInterpreter-Agent', 'agentResourceRoleArn': 'arn:aws:iam::735766051544:role/AmazonBRkExecutionRoleForAgents_VariantInterpreter-Agent', 'agentStatus': 'CREATING', 'createdAt': datetime.datetime(2025, 4, 26, 5, 19, 2, 775000, tzinfo=tzlocal()), 'description': 'Agent for interpreting annotated variant files and answering user Questions', 'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0', 'idleSession

# Agent Action Group

In [55]:
   agent_functions = [
    {
        'name': 'vep_feature_extraction',
        'description': 'retrieves the VEP annotated VCF features for a given patient id',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want a variant report",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'retrieve_existing_vep_report',
        'description': 'Starts with an existing VEP report for a given patient id. Returns summary of VEP report',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want to find a summarization of VEP report PDF",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [56]:
# Pause to make sure agent is created
# time.sleep(30)
# Now, we can configure and create an action group here:
try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
except:
    print("Action group already exists")
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
    print(agent_action_group_response)

In [57]:
# Create allow invoke permission on lambda
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId='allow_bedrock',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except: 
    print("Permission already exists")

{'ResponseMetadata': {'RequestId': '56936cb5-676e-484f-b5b9-776d60816c29', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 05:22:17 GMT', 'content-type': 'application/json', 'content-length': '366', 'connection': 'keep-alive', 'x-amzn-requestid': '56936cb5-676e-484f-b5b9-776d60816c29'}, 'RetryAttempts': 1}, 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:735766051544:function:VariantInterpreter-735766051544-Lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:735766051544:agent/BLUBEL66XB"}}}'}


In [ ]:
lambda_function_name

# Preparing Agent

In [58]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '6884cad5-b1ba-486d-b084-4b406c6eefaa', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 05:23:00 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '6884cad5-b1ba-486d-b084-4b406c6eefaa', 'x-amz-apigw-id': 'JnYUQHiHIAMElxQ=', 'x-amzn-trace-id': 'Root=1-680c6db4-307497c44a5d81c70456a970'}, 'RetryAttempts': 0}, 'agentId': 'BLUBEL66XB', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 4, 26, 5, 23, 0, 726820, tzinfo=tzlocal())}
